In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
load_dotenv()
PROJECT_ROOT = os.getenv('PROJECT_ROOT')

# Add PROJECT_ROOT to the Python path
import sys
sys.path.append(PROJECT_ROOT)



In [2]:
import pandas as pd

ORIGIN_ICAO = 'EGLL'
DEST_ICAO = 'UKBB'
ORIGIN_RWY = '09R'
DEST_RWY = '18R'

airports_df = pd.read_csv(os.path.join(PROJECT_ROOT, "data", "airac", "airports.csv"))


# Get the latitude and longitude of the origin and destination
origin_lat = airports_df[airports_df['icao'] == ORIGIN_ICAO]['latitude'].values[0]
origin_lon = airports_df[airports_df['icao'] == ORIGIN_ICAO]['longitude'].values[0]
dest_lat = airports_df[airports_df['icao'] == DEST_ICAO]['latitude'].values[0]
dest_lon = airports_df[airports_df['icao'] == DEST_ICAO]['longitude'].values[0]
# Origin and destination airport names
origin_name = airports_df[airports_df['icao'] == ORIGIN_ICAO]['name'].values[0]
dest_name = airports_df[airports_df['icao'] == DEST_ICAO]['name'].values[0]

print(f'Origin: {ORIGIN_ICAO} - {origin_name} ({origin_lat}, {origin_lon}, {ORIGIN_RWY})')
print(f'Destination: {DEST_ICAO} - {dest_name} ({dest_lat}, {dest_lon}, {DEST_RWY})')

Origin: EGLL - HEATHROW (51.4775, -0.461389, 09R)
Destination: UKBB - BORYSPIL INTL (50.344722, 30.893333, 18R)


In [3]:
from nav_graph import generate_navigraph, add_predecessor_access_for_graph

route_graph = generate_navigraph(ORIGIN_ICAO, DEST_ICAO, origin_lat, origin_lon, dest_lat, dest_lon,
                                  ORIGIN_RWY, DEST_RWY,
                                  w_dct=1.0, w_fra=1.0, w_proc=0.2)
add_predecessor_access_for_graph(route_graph)

Subsetting the ATS graph to the great circle path between origin and destination...


Adding edges to subset: 100%|██████████| 25011/25011 [00:00<00:00, 1885672.59it/s]


ATS graph loaded. Nodes: 1670, edges: 3817
Building FRA routing options...
Found 1037 FRA points within 100nm of the great circle path between origin and destination.          
Merging these into the ATS graph...
223 FRA points renamed for BALTIC & FRAIT & SECSI & SEE FRA


BALTIC & FRAIT & SECSI & SEE FRA: 100%|██████████| 267/267 [00:00<00:00, 1448.59it/s]


35 FRA points renamed for BELFRA


...

UKNESFRA: 100%|██████████| 133/133 [00:00<00:00, 2253.32it/s]


FRA graph merged into ATS graph. Nodes: 2667, edges: 121054
Computing cost for the ATS-FRA route graph...


Adding edges to subset: 100%|██████████| 121054/121054 [00:00<00:00, 814579.66it/s]


Adding SID and STAR graphs to the subset...
Route graph subset created. Nodes: 2923, edges: 121334
Great circle distance between origin and destination: 1179.84 nm


Processing predecessors list: 100%|██████████| 2923/2923 [00:00<00:00, 390581.11it/s]


In [4]:
import networkx as nx
from utils.flightplans import format_flightplan, get_detailed_flightplan_from_waypoint_list
from utils.haversine import haversine_distance

# Find the shortest path between origin and destination
origin_node = f"{ORIGIN_ICAO}_{ORIGIN_RWY}"
dest_node = f"{DEST_ICAO}_{DEST_RWY}"

# Check if the nodes exist in the graph
if origin_node not in route_graph.nodes:
    print(f"Origin node {origin_node} not found in graph. Available nodes for {ORIGIN_ICAO}:")
    for node in route_graph.nodes:
        if node.startswith(ORIGIN_ICAO):
            print(f"  {node}")
    # Try to find an alternative
    for node in route_graph.nodes:
        if node.startswith(ORIGIN_ICAO):
            origin_node = node
            print(f"Using {origin_node} as origin node instead")
            break

if dest_node not in route_graph.nodes:
    print(f"Destination node {dest_node} not found in graph. Available nodes for {DEST_ICAO}:")
    for node in route_graph.nodes:
        if node.startswith(DEST_ICAO):
            print(f"  {node}")
    # Try to find an alternative
    for node in route_graph.nodes:
        if node.startswith(DEST_ICAO):
            dest_node = node
            print(f"Using {dest_node} as destination node instead")
            break

# Find the shortest path
try:
    shortest_path = nx.shortest_path(route_graph, source=origin_node, target=dest_node, weight='cost')
    print(f"Shortest path found with {len(shortest_path)} waypoints")
    
    
    result = get_detailed_flightplan_from_waypoint_list(route_graph, shortest_path)

    # Print the flight plan
    print(format_flightplan(result))

except nx.NetworkXNoPath:
    print(f"No path found between {origin_node} and {dest_node}")
except Exception as e:
    print(f"Error finding path: {e}")

Origin node EGLL_09R not found in graph. Available nodes for EGLL:
  EGLL
Using EGLL as origin node instead
Destination node UKBB_18R not found in graph. Available nodes for UKBB:
  UKBB
Using UKBB as destination node instead
Shortest path found with 26 waypoints
EGLL CPT5J WOD BPK Q295 BRAIN M197 REDFA DERAM L980 POLON M70 OKROT SLV SLV2J UKBB


In [5]:
print(" ".join(shortest_path))

EGLL CPT5J_D130B CPT5J_D253K CPT5J_WOD WOD BPK TOTRI MATCH BRAIN GASBA RATLO REDFA ISMEF HLZ ARSAP DERAM POLON SOMOX TOLPA OKROT SLV SLV2J_SLV SLV2J_SL32B SLV2J_SLV50 SLV2J_D266B UKBB


# MCMC Sampling with Metropolis-Hastings

In [6]:
from planner_pivot import mcmc_step
MAX_ITER = 100
BURN_IN = 5_000 # can go as high as 10_000
THINNING = 25 # can go as high as 50 
sampled_routes = []
temperature = 10

total_accepted = 0

route = shortest_path

In [ ]:
import time 
start_time = time.time()
for i in range(MAX_ITER):
    new_route, accepted = mcmc_step(route_graph, route, temperature, verbose = False,
                                    max_depth=8)
    
    print(f'{i < BURN_IN and "Burn-in" or "Sampling"} | Iteration {i+1}, accepted: {total_accepted}               ', end='\r')

    if i < BURN_IN:
        continue
    
    if accepted:
        route = new_route
        total_accepted += 1
        
    if i % THINNING == 0:
        sampled_routes.append(route)
        
print(f"Total chain time: {time.time() - start_time:.2f} seconds")


# Splicer Library MCMC Sampling

In [7]:
# Add splicer library to the path
import sys
sys.path.append(os.path.join(PROJECT_ROOT, "lateral", "splicer"))
import splicer


In [8]:
import networkx as nx

def networkx_to_splicer(nx_graph):
    """
    Convert a networkx graph with string waypoint names into a splicer Graph 
    (using integer IDs). Assumes each node has 'lat' and 'lon' attributes.
    
    Parameters:
      nx_graph (networkx.Graph): A NetworkX graph with nodes as strings and 
                                 attributes 'lat' and 'lon'. Edge distances are
                                 taken from the 'weight' attribute, or default to 1.0.
    
    Returns:
      splicer_graph (planner.Graph): A splicer graph with integer-based node IDs.
      name_to_id (dict): Mapping from the original string waypoint names to integer IDs.
    """
    splicer_graph = splicer.Graph()
    name_to_id = {}
    
    # Assign an integer ID for each node in the networkx graph.
    for idx, node in enumerate(nx_graph.nodes()):
        name_to_id[node] = idx
        # Retrieve coordinates from node attributes; default to (0.0, 0.0) if not provided.
        node_attrs = nx_graph.nodes[node]
        lat = node_attrs.get("lat", 0.0)
        lon = node_attrs.get("lon", 0.0)
        splicer_graph.add_node(idx, lat, lon)
    
    # Add edges to the splicer graph using the new integer IDs.
    for u, v, data in nx_graph.edges(data=True):
        # Use the 'weight' attribute for the distance if available; default to 1.0.
        distance = data.get("weight", 1.0)
        splicer_graph.add_edge(name_to_id[u], name_to_id[v], distance)
    
    return splicer_graph, name_to_id


In [9]:
# Convert the networkx graph to a splicer graph.
splicer_graph, name_to_id = networkx_to_splicer(route_graph)

print("Mapping from waypoint name to integer ID:")
for name, node_id in list(name_to_id.items())[:5]:
    print(f"  {name} -> {node_id}")

# Now you can use `splicer_graph` with the MCMC functions.
# For example, create an initial route using integer IDs:
initial_route = [name_to_id[name] for name in shortest_path]
print("Initial route (integer IDs):", initial_route)

Mapping from waypoint name to integer ID:
  AMEPU -> 0
  ROS -> 1
  KL -> 2
  BAKAM -> 3
  NEPUT -> 4
Initial route (integer IDs): [2667, 2670, 2671, 2672, 323, 122, 1223, 1086, 650, 640, 651, 381, 2431, 528, 1534, 538, 539, 693, 694, 695, 696, 2869, 2870, 2871, 2872, 2668]


In [10]:
sampled_routes, final_route, total_accepted, acceptance_rate = splicer.start_mcmc(
    splicer_graph,
    initial_route,
    temperature=10.0,
    max_iter=MAX_ITER,
    burn_in=BURN_IN,
    thinning=THINNING,
    verbose=False
)

In [ ]:
# Create a reverse mapping from ID to waypoint name
id_to_name = {node_id: name for name, node_id in name_to_id.items()}

# Convert the sampled routes back to waypoint names
sampled_routes_names = []
for route in sampled_routes:
    route_names = [id_to_name[node_id] for node_id in route]
    sampled_routes_names.append(route_names)

# Print the first few sampled routes with waypoint names
print("\nSample of routes (waypoint names):")
for i, route in enumerate(sampled_routes_names[:3]):
    print(f"Route {i+1}: {route}")
    
# Convert sampled routes to detailed flight plans
print("\nConverting sampled routes to detailed flight plans...")

detailed_flightplans = []
for i, route in enumerate(sampled_routes_names):
    try:
        # Get detailed flight plan from waypoint list
        detailed_plan = get_detailed_flightplan_from_waypoint_list(route_graph, route)
        # Format the flight plan
        formatted_plan = format_flightplan(detailed_plan)
        detailed_flightplans.append(formatted_plan)
    except Exception as e:
        print(f"Error processing route {i+1}: {e}")

# Print a few example flight plans
print("\nSample of formatted flight plans:")
for i, plan in enumerate(detailed_flightplans[:3]):
    print(f"\nFlight Plan {i+1}:")
    print(plan)


# Print statistics
print(f"\nTotal routes sampled: {len(sampled_routes)}")
print(f"Acceptance rate: {acceptance_rate:.2f}")



# Histogram and Frequency Info

In [ ]:
# Create a histogram of flight plan frequencies
from collections import Counter
import matplotlib.pyplot as plt

# Count the frequency of each flight plan
flightplan_counter = Counter(detailed_flightplans)

# Get the top 20 most common flight plans
most_common = flightplan_counter.most_common(20)

# Extract data for plotting
plans = [f"Plan {i+1}" for i in range(len(most_common))]
frequencies = [count for _, count in most_common]

# Create the histogram
plt.figure(figsize=(12, 6))
plt.bar(plans, frequencies)
plt.xlabel('Flight Plans')
plt.ylabel('Frequency')
plt.title('Frequency of Top 20 Flight Plans')
plt.xticks(rotation=45)
plt.tight_layout()

# Print the most common flight plans with their frequencies
print("Top 5 most common flight plans:")
for i, (plan, count) in enumerate(most_common[:5]):
    print(f"\nPlan {i+1} (percentage: {count/len(detailed_flightplans)*100:.2f}%):")
    print(plan)

# Calculate and print some statistics
total_unique_plans = len(flightplan_counter)
print(f"\nTotal unique flight plans: {total_unique_plans}")
print(f"Most common flight plan appears {most_common[0][1]} times")
print(f"Percentage of flights using the most common plan: {most_common[0][1]/len(detailed_flightplans)*100:.2f}%")